In [1]:
!nvidia-smi

Sun Jan 17 11:17:24 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.27.04    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P8    10W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# *Installation*

In [2]:
!pip install tez
!pip install efficientnet_pytorch
!pip uninstall albumentations
!pip install albumentations

  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.0-cp36-none-any.whl size=16032 sha256=8bb1ce210dd9c23fa3be67beb8b795cab8a520d494695081fcdd842a6ed8f051
  Stored in directory: /root/.cache/pip/wheels/e9/c6/e1/7a808b26406239712cfce4b5ceeb67d9513ae32aa4b31445c6
Successfully built efficientnet-pytorch
Uninstalling albumentations-0.1.12:
  Would remove:
    /usr/local/lib/python3.6/dist-packages/albumentations-0.1.12.dist-info/*
    /usr/local/lib/python3.6/dist-packages/albumentations/*
Proceed (y/n)? y
  Successfully uninstalled albumentations-0.1.12
     |████████████████████████████████| 81kB 8.1MB/s 
     |████████████████████████████████| 37.6MB 82kB/s 
     |████████████████████████████████| 952kB 42.1MB/s 
  Found existing installation: imgaug 0.2.9
    Uninstalling imgaug-0.2.9:
      Successfully uninstalled imgaug-0.2.9


# imports

In [ ]:
import os
import albumentations
import  matplotlib.pyplot as plt
import pandas as pd
import cv2

import tez

from tez.datasets import ImageDataset
from tez.callbacks import EarlyStopping

import torch
import torch.nn as nn

import torchvision

from sklearn import metrics, model_selection
from efficientnet_pytorch import EfficientNet
from pathlib import Path
import argparse
import os

import albumentations
import pandas as pd
import numpy as np
import tez
import torch
import torch.nn as nn
from efficientnet_pytorch import EfficientNet
from sklearn import metrics, model_selection, preprocessing
from tez.callbacks import EarlyStopping
from tez.datasets import ImageDataset
from torch.nn import functional as F

from tqdm import tqdm
from pathlib import Path

from torch.utils.data import Dataset, DataLoader


%matplotlib inline

In [ ]:
# img = cv2.imread('/content/13735.webp')
# print(img.shape)
# cv2.imwrite(os.path.join('/content/drive/MyDrive', '13735.jpg'), img)
# cv2.imwrite(os.path.join(IMAGE_PATH, '13735.jpg'), img) 



In [ ]:
BASE_PATH = '/content/drive/MyDrive/upwork/img'
IMAGE_PATH = '/content/drive/MyDrive/upwork/img/data/fashion-dataset/images'
IMAGE_VECTOR_PATH = '/content/drive/MyDrive/upwork/img/data/fashion-dataset/image_vectors'

# Feature Extractor (using a pre-trained model)

## Model

In [ ]:
class FeatureExtractorEfficientNet(nn.Module):
    def __init__(self, efficientnet_model_name='efficientnet-b3'):
        super().__init__()
        self.effnet = EfficientNet.from_pretrained(efficientnet_model_name)
        self.out_feature_size = self.effnet._conv_head.out_channels

    def forward(self, image):
        batch_size, _, _, _ = image.shape

        x = self.effnet.extract_features(image)
        x = F.adaptive_avg_pool2d(x, 1).reshape(batch_size, -1)
        return x


## Dataloader

In [ ]:
class ImageVecDataset(Dataset):

    def __init__(self, IMAGE_PATH, aug=None):
        """
        Args:
            image_dir (string): Path to the images folder.
            aug: augumentation
        """
        self.image_dir = IMAGE_PATH
        self.images = list(Path(self.image_dir).glob('*.jpg'))
        if aug is None:
          self.aug = albumentations.Compose(
              [
                  albumentations.Normalize(
                      mean=[0.485, 0.456, 0.406],
                      std=[0.229, 0.224, 0.225],
                      max_pixel_value=255.0,
                      p=1.0,
                  ),
              ],
              p=1.0,
          )
        else:
          self.aug = aug

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
      filename = str(self.images[idx])
      # print(os.path.isfile(filename))

      try:
          img = cv2.imread(filename)
          img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
          img = cv2.resize(img,(256,256))
          img = torch.from_numpy(self.aug(image=img)['image']).type(torch.FloatTensor)
          img = img.permute(2, 0, 1)
      except:
          print(filename)
          img = torch.rand((3, 256, 256)).type(torch.FloatTensor)
        

      return str(self.images[idx].stem), img

## Feature Extraction

In [ ]:
image_path_list = list(Path(IMAGE_PATH).glob('*.jpg'))
print('Number of images: ', len(image_path_list))


fashion_dataset = ImageVecDataset(IMAGE_PATH)
fashion_dataloader = DataLoader(fashion_dataset, batch_size=8, shuffle=False, num_workers=4)

feature_extractor = FeatureExtractorEfficientNet()
feature_extractor.cuda()
feature_extractor.eval()

for i, (filenames, images) in tqdm(enumerate(fashion_dataloader), total=len(fashion_dataloader)):
  image_vector = feature_extractor(images.cuda()).cpu().detach().numpy()
  for i in range(len(images)):
      np.save(os.path.join(IMAGE_VECTOR_PATH, f'{filenames[i]}.npy'), image_vector[i])

Number of images:  29163


In [ ]:
# !rm -rf '/content/drive/MyDrive/upwork/img/data/fashion-dataset/image_vectors'

In [ ]:
!unzip '/content/drive/MyDrive/upwork/img/data/fashion-dataset/image_vectors.zip' -d '/content/drive/MyDrive/upwork/img/data/fashion-dataset/images/vectors'

Streaming output truncated to the last 5000 lines.
  inflating: /content/drive/MyDrive/upwork/img/data/fashion-dataset/images/vectors/image_vectors/39022.npy  
  inflating: /content/drive/MyDrive/upwork/img/data/fashion-dataset/images/vectors/image_vectors/39023.npy  
  inflating: /content/drive/MyDrive/upwork/img/data/fashion-dataset/images/vectors/image_vectors/39024.npy  
  inflating: /content/drive/MyDrive/upwork/img/data/fashion-dataset/images/vectors/image_vectors/39025.npy  
  inflating: /content/drive/MyDrive/upwork/img/data/fashion-dataset/images/vectors/image_vectors/39026.npy  
  inflating: /content/drive/MyDrive/upwork/img/data/fashion-dataset/images/vectors/image_vectors/39027.npy  
  inflating: /content/drive/MyDrive/upwork/img/data/fashion-dataset/images/vectors/image_vectors/39028.npy  
  inflating: /content/drive/MyDrive/upwork/img/data/fashion-dataset/images/vectors/image_vectors/39029.npy  
  inflating: /content/drive/MyDrive/upwork/img/data/fashion-dataset/images/ve

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/upwork/img/styles.csv', error_bad_lines=False)

b'Skipping line 6044: expected 10 fields, saw 11\nSkipping line 6569: expected 10 fields, saw 11\nSkipping line 7399: expected 10 fields, saw 11\nSkipping line 7939: expected 10 fields, saw 11\nSkipping line 9026: expected 10 fields, saw 11\nSkipping line 10264: expected 10 fields, saw 11\nSkipping line 10427: expected 10 fields, saw 11\nSkipping line 10905: expected 10 fields, saw 11\nSkipping line 11373: expected 10 fields, saw 11\nSkipping line 11945: expected 10 fields, saw 11\nSkipping line 14112: expected 10 fields, saw 11\nSkipping line 14532: expected 10 fields, saw 11\nSkipping line 15076: expected 10 fields, saw 12\nSkipping line 29906: expected 10 fields, saw 11\nSkipping line 31625: expected 10 fields, saw 11\nSkipping line 33020: expected 10 fields, saw 11\nSkipping line 35748: expected 10 fields, saw 11\nSkipping line 35962: expected 10 fields, saw 11\nSkipping line 37770: expected 10 fields, saw 11\nSkipping line 38105: expected 10 fields, saw 11\nSkipping line 38275: ex

In [ ]:
df.head()

,id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName
0,15970,Men,Apparel,Topwear,Shirts,Navy Blue,Fall,2011.0,Casual,Turtle Check Men Navy Blue Shirt
1,39386,Men,Apparel,Bottomwear,Jeans,Blue,Summer,2012.0,Casual,Peter England Men Party Blue Jeans
2,59263,Women,Accessories,Watches,Watches,Silver,Winter,2016.0,Casual,Titan Women Silver Watch
3,21379,Men,Apparel,Bottomwear,Track Pants,Black,Fall,2011.0,Casual,Manchester United Men Solid Black Track Pants
4,53759,Men,Apparel,Topwear,Tshirts,Grey,Summer,2012.0,Casual,Puma Men Grey T-shirt


In [ ]:
images = os.listdir(IMAGE_PATH)
count = 0
for i in df.id.tolist():
  if str(i) + '.jpg' not in images:
    count += 1
count

15281

In [ ]:
df.shape

(44424, 10)

In [ ]:
15281+29163

44444

In [ ]:
df.gender.unique()

array(['Men', 'Women', 'Boys', 'Girls', 'Unisex'], dtype=object)

In [ ]:
def encode_label(df, column):
  if column not in df.columns:
    print(f"column: {column} nor present in dataframe")
    le = preprocessing.LabelEncoder()
    le.fit(df[column].tolist())
    print("Number of classes: ", len(le.classes_))
    return le.transform(df[column].tolist())

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(df.gender.tolist())
le.classes_

array(['Boys', 'Girls', 'Men', 'Unisex', 'Women'], dtype='<U6')

In [ ]:
le.transform(df.gender.tolist())[:20]

array([2, 2, 4, 2, 2, 2, 2, 4, 2, 2, 2, 4, 2, 4, 0, 2, 2, 1, 4, 4])

In [ ]:
df.gender.tolist()[:20]

['Men',
 'Men',
 'Women',
 'Men',
 'Men',
 'Men',
 'Men',
 'Women',
 'Men',
 'Men',
 'Men',
 'Women',
 'Men',
 'Women',
 'Boys',
 'Men',
 'Men',
 'Girls',
 'Women',
 'Women']